In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip') as train:
    train.extractall('.')

with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip') as test:
    test.extractall('.')

In [ ]:
import tensorflow as tf

In [ ]:
# tf.config.experimental.list_physical_devices("GPU")

In [ ]:
filenames = os.listdir('/kaggle/working/train/')

In [ ]:
labels =[]
for i in filenames :
    labels.append(i.split('.')[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder 

In [ ]:
encoder = LabelEncoder()
encoder.fit(labels)

In [ ]:
y = encoder.transform(labels)

In [ ]:
y = y.astype(str)

In [ ]:
import random

In [ ]:
image = random.choice(filenames)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img , img_to_array

In [ ]:
f = load_img("/kaggle/working/train/"+image)
plt.imshow(f)
photo = img_to_array(f)
photo.shape

In [ ]:
file = {"names":filenames , "y":y}

In [ ]:
df = pd.DataFrame(file)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train ,y_train = train_test_split(df , test_size = .3 , random_state =42 )

In [ ]:
x_train

In [ ]:
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# train_datagen = ImageDataGenerator(rescale=1.0/255.0)


# valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
    

In [ ]:
# train_generator = train_datagen.flow_from_dataframe(
#     x_train,
#     "/kaggle/working/train/", 
#     x_col='names',
#     y_col='y',
# class_mode='binary', batch_size=64, target_size=(481, 287))

# valid_generator = valid_datagen.flow_from_dataframe( 
#     y_train,
#     "/kaggle/working/train/", 
#     x_col='names',
#     y_col='y',
#   class_mode='binary', batch_size=64, target_size=(481, 287))

In [ ]:
# vgg16 model used for transfer learning on the dogs and cats dataset
import sys
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
 
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model

# define model
model = define_model()
# create data generator
datagen = ImageDataGenerator(featurewise_center=True)
# specify imagenet mean values for centering
datagen.mean = [123.68, 116.779, 103.939]
# prepare iterator
train_it = datagen.flow_from_dataframe( x_train,
    "/kaggle/working/train/", 
    x_col='names',
    y_col='y',
		class_mode='binary', batch_size=64, target_size=(224, 224))
test_it = datagen.flow_from_dataframe( y_train,
    "/kaggle/working/train/", 
    x_col='names',
    y_col='y',
	class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=5)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))
	# learning curves
summarize_diagnostics(history)
 

In [ ]:
test_filenames = os.listdir("/kaggle/working/test1")

In [ ]:
test_filenames

test_df = pd.DataFrame({
    'filename': test_filenames,
})
test_df

In [ ]:
test_data = ImageDataGenerator(rescale=1./255)
test_data

In [ ]:
test_data_gen = test_data.flow_from_dataframe(
    test_df, 
    "/kaggle/working/test1", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(224, 224),
    batch_size=64,
    shuffle=False
)

In [ ]:
predictions = model.predict_generator(test_data_gen,2000)

In [ ]:
len(predictions)

In [ ]:
results = (predictions > 0.5).astype(int)

In [ ]:
results = pd.DataFrame(results)

In [ ]:
# submission = pd.read_csv('sampleSubmission.csv')

In [ ]:
results.columns = ['index']

In [ ]:
label = []
for i in results['index']:
    if i ==0:
        label.append('cat')
    else:
        label.append('dog')

In [ ]:
test_df['label'] =results['index']

In [ ]:
test_df.to_csv("submission.csv",index = False)